In [60]:
pip install yfinance

In [61]:
import yfinance as yf
data = yf.download("RELIANCE.NS", start="2018-01-01", end="2024-12-31")
data.to_csv("reliance_data.csv")


[*********************100%***********************]  1 of 1 completed


In [62]:
data

Price,Close,High,Low,Open,Volume
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Date,,,,,
2018-01-01,411.716949,417.577592,410.698687,417.577592,9453202
2018-01-02,412.350525,416.152046,410.200863,413.187768,9499419
2018-01-03,414.002350,419.071034,413.210361,418.618456,13507800
2018-01-04,416.491425,417.170276,414.409645,415.518417,9008932
2018-01-05,417.826477,419.478336,416.468804,417.170283,7441284
...,...,...,...,...,...
2024-12-23,1222.300049,1227.199951,1213.199951,1215.000000,10052824
2024-12-24,1222.750000,1233.550049,1221.000000,1222.300049,6734917


In [63]:
data

Price,Close,High,Low,Open,Volume
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Date,,,,,
2018-01-01,411.716949,417.577592,410.698687,417.577592,9453202
2018-01-02,412.350525,416.152046,410.200863,413.187768,9499419
2018-01-03,414.002350,419.071034,413.210361,418.618456,13507800
2018-01-04,416.491425,417.170276,414.409645,415.518417,9008932
2018-01-05,417.826477,419.478336,416.468804,417.170283,7441284
...,...,...,...,...,...
2024-12-23,1222.300049,1227.199951,1213.199951,1215.000000,10052824
2024-12-24,1222.750000,1233.550049,1221.000000,1222.300049,6734917


In [64]:
data=data.dropna()

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [66]:
pip install backtesting

Note: you may need to restart the kernel to use updated packages.


In [67]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG  # You can replace this with your own data
import pandas as pd
import numpy as np

# Indicator Functions

def bollinger_middle(data, period=20):
    return pd.Series(data).rolling(period).mean().values

def bollinger_upper(data, period=20, k=2):
    middle = pd.Series(data).rolling(period).mean()
    std = pd.Series(data).rolling(period).std()
    return (middle + k * std).values

def bollinger_lower(data, period=20, k=2):
    middle = pd.Series(data).rolling(period).mean()
    std = pd.Series(data).rolling(period).std()
    return (middle - k * std).values

def compute_rsi(data, period=14):
    delta = pd.Series(data).diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    loss = loss.replace(0, 0.001)  # Avoid division by zero
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.values

def compute_macd(data, fast=12, slow=26, signal=9):
    ema_fast = pd.Series(data).ewm(span=fast, adjust=False).mean()
    ema_slow = pd.Series(data).ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    histogram = macd_line - signal_line
    return macd_line.values, signal_line.values, histogram.values

def compute_atr(high, low, close, period=14):
    tr1 = pd.Series(high) - pd.Series(low)
    tr2 = abs(pd.Series(high) - pd.Series(close).shift())
    tr3 = abs(pd.Series(low) - pd.Series(close).shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(period).mean()
    return atr.values

# Strategy Class

class MeanReversionStrategy(Strategy):
    period = 20
    k = 2
    rsi_period = 14
    rsi_overbought = 75
    rsi_oversold = 25
    macd_fast = 12
    macd_slow = 26
    macd_signal = 9
    atr_period = 14
    trailing_stop = 1.5
    risk_pct = 0.02

    def init(self):
        # Bollinger Bands
        self.middle_band = self.I(bollinger_middle, self.data.Close, self.period)
        self.upper_band = self.I(bollinger_upper, self.data.Close, self.period, self.k)
        self.lower_band = self.I(bollinger_lower, self.data.Close, self.period, self.k)
        
        # RSI
        self.rsi = self.I(compute_rsi, self.data.Close, self.rsi_period)

        # MACD
        self.macd_line = self.I(lambda x: compute_macd(x, self.macd_fast, self.macd_slow, self.macd_signal)[0], self.data.Close)
        self.signal_line = self.I(lambda x: compute_macd(x, self.macd_fast, self.macd_slow, self.macd_signal)[1], self.data.Close)
        self.histogram = self.I(lambda x: compute_macd(x, self.macd_fast, self.macd_slow, self.macd_signal)[2], self.data.Close)

        # ATR
        self.atr = self.I(compute_atr, self.data.High, self.data.Low, self.data.Close, self.atr_period)
        self.atr_avg = self.I(lambda x: pd.Series(x).rolling(50).mean().values, self.atr)

        # Trend filter
        self.sma_trend = self.I(lambda x: pd.Series(x).rolling(50).mean().values, self.data.Close)

    def next(self):
        price = self.data.Close[-1]
        rsi = self.rsi[-1]
        lower_band = self.lower_band[-1]
        upper_band = self.upper_band[-1]
        middle_band = self.middle_band[-1]
        macd_line = self.macd_line[-1]
        signal_line = self.signal_line[-1]
        atr = self.atr[-1]
        atr_avg = self.atr_avg[-1]
        is_uptrend = price > self.sma_trend[-1] * 1.01
        is_downtrend = price < self.sma_trend[-1] * 0.99

        if atr < atr_avg * 1.2:
            if not self.position:
                if (price < lower_band and rsi < self.rsi_oversold) or (
                    macd_line > signal_line and self.macd_line[-2] <= self.signal_line[-2] and rsi < 55 and is_uptrend
                ):
                    stop_loss = price - (self.trailing_stop * atr)
                    take_profit = price + (2 * atr)
                    size = (self.equity * self.risk_pct) / (self.trailing_stop * atr)
                    size = max(1, int(size))
                    self.buy(sl=stop_loss, tp=take_profit, size=size)

                elif (price > upper_band and rsi > self.rsi_overbought) or (
                    macd_line < signal_line and self.macd_line[-2] >= self.signal_line[-2] and rsi > 45 and is_downtrend
                ):
                    stop_loss = price + (self.trailing_stop * atr)
                    take_profit = price - (2 * atr)
                    size = (self.equity * self.risk_pct) / (self.trailing_stop * atr)
                    size = max(1, int(size))
                    self.sell(sl=stop_loss, tp=take_profit, size=size)
            else:
                if self.position.is_long and crossover(self.data.Close, self.middle_band):
                    self.position.close()
                elif self.position.is_short and crossover(self.middle_band, self.data.Close):
                    self.position.close()

                for trade in self.trades:
                    if len(self.data.Close) - trade.entry_bar >= 10:
                        trade.close()

# Run the backtest
bt = Backtest(GOOG, MeanReversionStrategy, cash=10000, commission=0.002)
stats = bt.run()

print("Backtest Statistics:")
print(stats)

bt.plot()


Backtest.run:   0%|          | 0/2085 [00:00<?, ?bar/s]

Backtest Statistics:
Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    12.05773
Equity Final [$]                  10795.38733
Equity Peak [$]                   11186.75312
Commissions [$]                    1242.07302
Return [%]                            7.95387
Buy & Hold Return [%]               367.24817
Return (Ann.) [%]                     0.90193
Volatility (Ann.) [%]                 5.05112
CAGR [%]                              0.62087
Sharpe Ratio                          0.17856
Sortino Ratio                          0.2716
Calmar Ratio                          0.05292
Alpha [%]                             8.48593
Beta                                 -0.00145
Max. Drawdown [%]                   -17.04322
Avg. Drawdown [%]                    -5.24877
Max. Drawdown Duration     1801 days 00:00:00
Avg. Drawdown Duration      402 days 00:00:00
# Trades     

GridPlot(id='p2879', ...)